In [1]:
from dotenv import load_dotenv

import os

load_dotenv()

True

## Construção de Tools 

A construção de Tools com o objeto ``Structuredtool`` é importante devido a sua capacidade de ser compatível com as cadeias do ``Langchain``. Isso promove uma série de benefícios, como suporte para ``paralelização``, ``reutilização``, ``observabilidade``, ``monitoramento`` e outros.

Para a construção das ferramentas, podemos utilizar o código abaixo, o exemplo que vamos utilizar será reutilizado para construir uma ferramenta de agente que irá pesquisar tendências via ``Google Trends`` e retornará uma lista de conteúdos que estão em alta.

In [2]:
# Importar a biblioteca necessária
from pytrends.request import TrendReq
from langchain_core.tools import StructuredTool

# Inicializar um objeto TrendReq
pytrends = TrendReq(hl='pt-BR', tz=360)

def trends_per_country(country: str, head: int) -> str:
    """trends per country"""
    country = country.lower()
    daily_trending_searches = pytrends.trending_searches(pn=country)
    word_list = daily_trending_searches.head(head).values.T[0]
    return "\n".join(word_list)


async def atrends_per_country(country: str, head: int) -> str:
    """trends per country"""
    country = country.lower()
    daily_trending_searches = pytrends.trending_searches(pn=country)
    word_list = daily_trending_searches.head(head).values.T[0]
    return "\n".join(word_list)

def create_trends_tool():
    return StructuredTool.from_function(func=trends_per_country, coroutine=atrends_per_country)

trends = create_trends_tool()

print(trends.invoke({"country": "brazil", "head": 10}))
print("--------------------------------")
print(
    await trends.ainvoke({"country": "united_states", "head": 10})
)  # Uses use provided amultiply without additional overhead


Campeonato Carioca
Resultado TSE Unificado
Cruzeiro x São Paulo
Paulista
Copa do Nordeste
Delegado morto
XRP
Inter de Milão
Bayern
Palmeiras x Portuguesa
--------------------------------
Pam Bondi
CPI report
Red dye 3
Coco Gauff
Premier League
ASEAN
XRP
Starbucks reverses open-door policy
Greater Los Angeles wildfires
Jessica Simpson


## Construção de Tools com o Serper

Aqui vamos construir outra ferramenta que irá pesquisar as tendências geradas pelo pytrends da etapa anterior. Abaixo você vai encontrar uma função que recebe uma lista de queries e retorna uma lista de notícias que foram retornadas pelo ``GoogleSerperAPIWrapper`` (Pesquise sobre o Serper para entender melhor).

Por fim, vamos acompanhar o código que será utilizado para construir a ferramenta.

In [3]:
from typing import List
import os

from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from tqdm import tqdm

def get_serper_results(queries: List[str],
                       k: int=5,
                       type_content: str='news',
                       hl: str='pt',
                       gl: str='br') -> List[dict]:
    
    """
    Get the multiple search results from Google Serper API
    
    Args:
    - queries: List of queries to be searched
    - k: Number of results to be returned
    - type_content: Type of content to be searched
    - hl: Language to be searched
    - gl: Country to be searched
    
    Returns:
    - List of results from the search
    
    """

    search = GoogleSerperAPIWrapper(gl=gl,
                                    hl=hl,
                                    k=k, 
                                    type=type_content) # type: Literal['news', 'search', 'places', 'images'] = 'search'
    
    results = [search.results(query) for query in queries]
    return results

def get_serper_with_scrapping(queries: List[str],
                              k: int=5,
                              type_content: str='news',
                              hl: str='pt',
                              gl: str='br') -> List[dict]:
    
    """
    Get the multiple search results from Google Serper API and scrap the content
    
    Args:
    - queries: List of queries to be searched
    - k: Number of results to be returned
    - type_content: Type of content to be searched
    - hl: Language to be searched
    - gl: Country to be searched
    
    Returns:
    - List of results from the search
    
    """
    
    results = get_serper_results(queries, 
                                 k, 
                                 type_content, 
                                 hl, 
                                 gl)
    
    dict_results_news = {}
    for r in tqdm(results):
        q = r['searchParameters']['q']
        news = r['news']
        
        for i in range(len(news)):
            n = news[i]
            link = n['link']
            loader = WebBaseLoader(web_paths=[link])
            docs = loader.load()
            news[i]['content'] = "\n".join([x.page_content for x in docs])
        
        dict_results_news[q] = news
        
    return dict_results_news

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
def to_list(text):
    return text.split("\n")

trends_text = trends.invoke({"country": "brazil", 
                             "head": 3})
trends_list = to_list(trends_text)
trends_list

['Campeonato Carioca', 'Resultado TSE Unificado', 'Cruzeiro x São Paulo']

In [5]:
contents_news = get_serper_with_scrapping(trends_list, k=5)

100%|██████████| 3/3 [00:18<00:00,  6.22s/it]


In [6]:
contents_news

{'Campeonato Carioca': [{'title': 'Botafogo mostra evolução e vence a Portuguesa no Campeonato Carioca',
   'link': 'https://odia.ig.com.br/esporte/botafogo/2025/01/6985875-botafogo-mostra-evolucao-e-vence-a-portuguesa-no-campeonato-carioca.html',
   'section': 'Botafogo bate a Portuguesa e vence a primeira no Carioca',
   'date': 'há 17 horas',
   'source': 'O Dia',
   'position': 1,
   'content': "\n\n\n\n\n\n\n\nBotafogo mostra evolução e vence a Portuguesa no Campeonato Carioca | Botafogo | O Dia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMatéria Salva!\nAgora você pode ler esta notícia off-line\n\n\n\n\n                Matéria removida da seção links salvos\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAssine\n\n\nAcessar edição digital\n\n\n\n\n\n\n\n\n\n\nBuscar no O 

## Conclusão:

Em primeiro lugar, é possível observar a utilidade de construir ferramentas (Tools). Elas são capazes de facilitar a reutilização de código, a paralelização de tarefas, a observabilidade e o monitoramento de tarefas. Por fim, vamos utilizar essas ferramentas para construir um agente que irá pesquisar tendências via ``Google Trends`` e retornar uma lista de conteúdos que estão em alta, a partir de pesquisas na web.